In [2]:
import pandas as pd
from sqlalchemy import create_engine

## Import CSV as a DataFrame

In [3]:
filepath = "Resources/DB-Vehicle-PTE.csv"
cars_df = pd.read_csv(filepath, encoding="ISO-8859-1")
cars_df.head()

,Brand,Vehicle,Engine,Engine code,Torque (Nm/rpm),Power (hp - kW /rpm),Average fuel consumption (l/100 km),CO2 (g/km),Cx/S(m²)/SCx,Weight(3p/5p) kg,Displacement,Block material /cylinder head,Compression ratio,GearBox type,Tyre,Others,Traction type
0,Renault,Twingo,1.2L 60 hp,D7F,93/2500,58-43/5250,5.6,130,?,1000,1149,cast iron /aluminium,9.6,MT,165/65 R14,12V,2wd
1,Renault,Twingo,1.2L 75 hp,D4F,108/4250,75-55/5500,5.1,120,?,1025,1149,cast iron /aluminium,9.8,MT,165/65 R14,16V,2wd
2,Renault,Twingo,1.5L dCi 65 hp,K9KBP,160/1900,65-48/3750,4.3,113,?,1055,1461,cast iron /aluminium,17.9,MT,165/65 R14,8V+TC,2wd
3,Renault,Twingo,1.2L 75 hp AT,D4F,108/4250,75-55/5500,5.6,132,?,1030,1149,cast iron /aluminium,9.8,AT 5,165/65 R14,16V,2wd
4,Renault,Twingo,1.5L dCi 85 hp,K9KHP,200/1750,84-62/3750,4,104,?,1055,1461,cast iron /aluminium,17.9,MT,165/65 R14,8V+TC+CAC,2wd


## Create new dataframe with columns to keep

In [4]:
cleaned_cars_df = cars_df[['Brand', 'Vehicle', 'Engine', 'Power (hp - kW /rpm)', 'Average fuel consumption (l/100 km)', 'CO2 (g/km)', 'Weight(3p/5p) kg', 'Block material /cylinder head', 'Traction type']]
cleaned_cars_df.head()

,Brand,Vehicle,Engine,Power (hp - kW /rpm),Average fuel consumption (l/100 km),Average fuel consumption (l/100 km),CO2 (g/km),Weight(3p/5p) kg,Block material /cylinder head,Traction type
0,Renault,Twingo,1.2L 60 hp,58-43/5250,5.6,5.6,130,1000,cast iron /aluminium,2wd
1,Renault,Twingo,1.2L 75 hp,75-55/5500,5.1,5.1,120,1025,cast iron /aluminium,2wd
2,Renault,Twingo,1.5L dCi 65 hp,65-48/3750,4.3,4.3,113,1055,cast iron /aluminium,2wd
3,Renault,Twingo,1.2L 75 hp AT,75-55/5500,5.6,5.6,132,1030,cast iron /aluminium,2wd
4,Renault,Twingo,1.5L dCi 85 hp,84-62/3750,4,4,104,1055,cast iron /aluminium,2wd


## Rename columns

In [5]:
cleaned_cars_df = cleaned_cars_df.rename(columns={'Brand': 'brand',
                                                  'Vehicle': 'vehicle', 
                                                  'Engine': 'engine',
                                                  'Power (hp - kW /rpm)': 'power',
                                                 'Average fuel consumption (l/100 km)': 'average_fuel_consumption',
                                                 'CO2 (g/km)': 'co2',
                                                 'Weight(3p/5p) kg': 'weight', 
                                                  'Block material /cylinder head': 'material', 
                                                  'Traction type': 'traction_type'})
cleaned_cars_df.head()

,brand,vehicle,engine,power,average_fuel_consumption,average_fuel_consumption,co2,weight,material,traction_type
0,Renault,Twingo,1.2L 60 hp,58-43/5250,5.6,5.6,130,1000,cast iron /aluminium,2wd
1,Renault,Twingo,1.2L 75 hp,75-55/5500,5.1,5.1,120,1025,cast iron /aluminium,2wd
2,Renault,Twingo,1.5L dCi 65 hp,65-48/3750,4.3,4.3,113,1055,cast iron /aluminium,2wd
3,Renault,Twingo,1.2L 75 hp AT,75-55/5500,5.6,5.6,132,1030,cast iron /aluminium,2wd
4,Renault,Twingo,1.5L dCi 85 hp,84-62/3750,4,4,104,1055,cast iron /aluminium,2wd
